### Carga de dependencias

In [2]:
import pandas as pd
import numpy as np
import uuid
from datetime import datetime, timedelta
import random

#### Se construye una función que permite:

* Calcular cuántos días hay entre start_date y end_date usando la diferencia de fechas. En resumen, el rango de fechas solicitado.
* Tener una lista con todas las fechas
* Poner a disposición unos pesos para cumplir con el incremental de registros hacia fechas más recientes que pide el ejercicio.
* Aplicar un pesos adicional a las fechas de diciembre (estacionalidad de ventas)
* Escoger filas más recientes y de diciembre como filas, asignando mayor probabilidad

In [7]:
def generate_sales_data(n_rows=50000):
    # Semilla config
    np.random.seed(42)
    random.seed(42)
    
    # fechas con estacionalidad y tendencia creciente
    start_date = datetime(2023, 1, 1)
    end_date = datetime(2024, 12, 31)
    
    # fechas con más peso hacia 2024
    days = (end_date - start_date).days
    base_dates = [start_date + timedelta(days=x) for x in range(days)]
    weights = np.linspace(1, 2, len(base_dates))  # Tendencia creciente
    # estacionalidad en diciembre)
    for i, date in enumerate(base_dates):
        if date.month == 12:
            weights[i] *= 1.5
    
    dates = random.choices(base_dates, weights=weights, k=n_rows)
    
    # datos de base
    data = {
        'order_id': [str(uuid.uuid4()) for _ in range(n_rows)],
        'customer_id': pd.Series(np.random.randint(1, 10_001, n_rows), dtype='Int64'), 
        'product_id': pd.Series(np.random.randint(1, 1_001, n_rows), dtype='Int64'),    
        'quantity': pd.Series(np.random.randint(1, 21, n_rows), dtype='Int64'),         
        'price': np.random.uniform(1.0, 500.0, n_rows),
        'order_date': dates,
        'region': np.random.choice(['North', 'South', 'East', 'West'], n_rows)
    }
    
    # relacion discount inversa con price
    max_price = max(data['price'])
    data['discount'] = [(1 - p/max_price) * 0.3 * random.uniform(0.8, 1.2) for p in data['price']]
    data['discount'] = [min(max(d, 0.0), 0.3) for d in data['discount']]
    
    # shipping_priority basado en region
    priority_map = {
        'North': ['High'] * 50 + ['Medium'] * 30 + ['Low'] * 20,
        'South': ['High'] * 20 + ['Medium'] * 50 + ['Low'] * 30,
        'East': ['High'] * 30 + ['Medium'] * 40 + ['Low'] * 30,
        'West': ['High'] * 25 + ['Medium'] * 45 + ['Low'] * 30
    }
    
    data['shipping_priority'] = [random.choice(priority_map[region]) 
                               for region in data['region']]
    
    # convertir a DataFrame
    df = pd.DataFrame(data)
    
    # ruido y valores faltantes (5% de las filas)
    rows_to_noise = int(n_rows * 0.05)
    noise_indices = np.random.choice(n_rows, rows_to_noise, replace=False)
    
    for idx in noise_indices:
        # seleccion de tres columnas aleatorias (excluyendo order_id)
        columns_to_noise = np.random.choice(
            [col for col in df.columns if col != 'order_id'], 
            3, 
            replace=False
        )
        
        for col in columns_to_noise:
            noise_type = random.choice(['null', 'extreme', 'remove'])
            
            if noise_type == 'null' or noise_type == 'remove':
                df.at[idx, col] = pd.NA if col in ['customer_id', 'product_id', 'quantity'] else np.nan
            elif noise_type == 'extreme':
                if col in ['price', 'discount']:
                    df.at[idx, col] = -9999.99
                elif col in ['customer_id', 'product_id', 'quantity']:
                    df.at[idx, col] = -9999
                elif col == 'order_date':
                    df.at[idx, col] = pd.NaT
                elif col == 'region' or col == 'shipping_priority':
                    df.at[idx, col] = 'UNKNOWN'

    return df

In [ ]:
# Genero el dataset
df = generate_sales_data()

In [ ]:
#Le hago un head para chequear
df.head()

,order_id,customer_id,product_id,quantity,price,order_date,region,discount,shipping_priority
0,249093af-4bed-430f-a55d-73e8dd782870,7271,923,10,410.048837,2024-06-07,East,0.064040,Low
1,8da511f7-ef36-4a18-9b31-1b107876ce28,861,621,20,466.519110,2023-01-29,East,0.021410,Medium
2,db3b8b9c-4917-4ec5-a7df-1c661aa6b69c,5391,677,3,35.175263,2023-09-24,West,0.243724,Medium
3,4aeec22a-d9bb-429c-9581-46068d89578a,5192,370,9,75.551426,2023-08-11,West,0.272166,Low
4,1224d200-8fe1-4c9f-9ad2-546613fe1cc2,<NA>,<NA>,15,61.812616,2024-08-09,NaN,0.233102,High


In [10]:
#el dataset se ve bien, ahora un describe
df.describe()

,customer_id,product_id,quantity,price,order_date,discount
count,49391.0,49383.0,49377.0,49334.000000,49069,49380.000000
mean,4887.314855,438.350201,-54.588594,185.752157,2024-02-16 23:32:14.296602624,-63.237726
min,-9999.0,-9999.0,-9999.0,-9999.990000,2023-01-01 00:00:00,-9999.990000
25%,2429.0,244.0,5.0,122.821126,2023-09-05 00:00:00,0.072768
50%,4949.0,495.0,10.0,247.267711,2024-03-01 00:00:00,0.147068
75%,7470.0,747.0,15.0,373.511021,2024-08-17 00:00:00,0.221303
max,10000.0,1000.0,20.0,499.992113,2024-12-30 00:00:00,0.300000
std,3129.617508,841.751689,804.453616,821.329144,NaN,793.644990


La información hasta aquí cumple con lo solicitado, en el siguiente paso se profundiza sobre la estructura del df.

In [11]:
import os

print(os.getcwd())

c:\Users\carlo\Documents\dm_prueba_t-cnica\notebooks


In [12]:
os.chdir('C:/Users/carlo/Documents/dm_prueba_t-cnica/')

Se guardan los datos sintéticos en carpeta data

In [13]:
df.to_csv('data/raw_sales_data.csv',sep=";", index=False)